In [15]:
import findspark
findspark.init()  # Initialize findspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("PySpark for Retail_case_study") \
    .getOrCreate()

# Verify SparkSession is created
print(spark)

In [28]:
import pandas as pd

In [23]:
from pyspark.sql.functions import to_date

In [41]:
from pyspark.sql.types import *





customer_df = spark.read.csv(path='/home/hdoop/Docs/spark_files/case_study_analysis/Customer.csv',header=True,inferSchema = True)

In [42]:
customer_df.show(n=2)

+-----------+------+------+---------+
|customer_Id|   DOB|Gender|city_code|
+-----------+------+------+---------+
|     268408|2-1-70|     M|        4|
|     269696|7-1-70|     F|        8|
+-----------+------+------+---------+
only showing top 2 rows



In [43]:
customer_df.printSchema()

root
 |-- customer_Id: integer (nullable = true)
 |-- DOB: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: integer (nullable = true)



In [44]:
customer_df = customer_df.withColumn("DOB",to_date(customer_df.DOB))

In [45]:
customer_df.printSchema()

root
 |-- customer_Id: integer (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: integer (nullable = true)



In [46]:
customer_df.show()

+-----------+----+------+---------+
|customer_Id| DOB|Gender|city_code|
+-----------+----+------+---------+
|     268408|NULL|     M|        4|
|     269696|NULL|     F|        8|
|     268159|NULL|     F|        8|
|     270181|NULL|     F|        2|
|     268073|NULL|     M|        1|
|     273216|NULL|     F|        5|
|     266929|NULL|     M|        8|
|     275152|NULL|     M|        4|
|     275034|NULL|     F|        4|
|     273966|NULL|     M|        8|
|     270829|NULL|     F|        8|
|     266997|NULL|     M|        9|
|     272455|NULL|     M|        7|
|     274593|NULL|     M|       10|
|     273391|NULL|     F|        2|
|     267657|NULL|     F|        7|
|     271344|NULL|     F|        5|
|     268275|NULL|     F|        7|
|     274630|NULL|     M|        4|
|     273541|NULL|     M|        4|
+-----------+----+------+---------+
only showing top 20 rows



In [12]:
customer_df = customer_df.withColumn("city_code",customer_df.city_code.cast('Integer'))

In [14]:
customer_df.printSchema()

root
 |-- customer_Id: string (nullable = true)
 |-- DOB: date (nullable = true)
 |-- Gender: string (nullable = true)
 |-- city_code: integer (nullable = true)



In [18]:
product_heirarchy = spark.read.csv(path ='/home/hdoop/Docs/spark_files/case_study_analysis/prod_cat_info.csv',header=True,inferSchema=True)

In [19]:
product_heirarchy.show()

+-------------+----------------+-----------------+-------------------+
|prod_cat_code|        prod_cat|prod_sub_cat_code|        prod_subcat|
+-------------+----------------+-----------------+-------------------+
|            1|        Clothing|                4|               Mens|
|            1|        Clothing|                1|              Women|
|            1|        Clothing|                3|               Kids|
|            2|        Footwear|                1|               Mens|
|            2|        Footwear|                3|              Women|
|            2|        Footwear|                4|               Kids|
|            3|     Electronics|                4|            Mobiles|
|            3|     Electronics|                5|          Computers|
|            3|     Electronics|                8|Personal Appliances|
|            3|     Electronics|                9|            Cameras|
|            3|     Electronics|               10|    Audio and video|
|     

In [20]:
product_heirarchy.printSchema()

root
 |-- prod_cat_code: integer (nullable = true)
 |-- prod_cat: string (nullable = true)
 |-- prod_sub_cat_code: integer (nullable = true)
 |-- prod_subcat: string (nullable = true)



In [21]:
transaction =spark.read.csv(path = '/home/hdoop/Docs/spark_files/case_study_analysis/Transactions.csv',header = True,inferSchema =True)

In [22]:
transaction.show()
transaction.printSchema()

+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+--------------+
|transaction_id|cust_id| tran_date|prod_subcat_code|prod_cat_code|Qty| Rate|    Tax|total_amt|    Store_type|
+--------------+-------+----------+----------------+-------------+---+-----+-------+---------+--------------+
|   80712190438| 270351|28-02-2014|               1|            1| -5| -772|  405.3|  -4265.3|        e-Shop|
|   29258453508| 270384|27-02-2014|               5|            3| -5|-1497|785.925|-8270.925|        e-Shop|
|   51750724947| 273420|24-02-2014|               6|            5| -2| -791| 166.11| -1748.11|      TeleShop|
|   93274880719| 271509|24-02-2014|              11|            6| -3|-1363|429.345|-4518.345|        e-Shop|
|   51750724947| 273420|23-02-2014|               6|            5| -2| -791| 166.11| -1748.11|      TeleShop|
|   97439039119| 272357|23-02-2014|               8|            3| -2| -824| 173.04| -1821.04|      TeleShop|
|   456498

In [24]:
transaction = transaction.withColumn("tran_date",to_date(transaction.tran_date))

In [25]:
transaction.printSchema()

root
 |-- transaction_id: long (nullable = true)
 |-- cust_id: integer (nullable = true)
 |-- tran_date: date (nullable = true)
 |-- prod_subcat_code: integer (nullable = true)
 |-- prod_cat_code: integer (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- Rate: integer (nullable = true)
 |-- Tax: double (nullable = true)
 |-- total_amt: double (nullable = true)
 |-- Store_type: string (nullable = true)



In [30]:
customer_trans=customer_df.join(transaction,[customer_df.customer_Id == transaction.cust_id],"inner")

In [33]:
customer_df.show()

+-----------+----+------+---------+
|customer_Id| DOB|Gender|city_code|
+-----------+----+------+---------+
|     268408|NULL|     M|        4|
|     269696|NULL|     F|        8|
|     268159|NULL|     F|        8|
|     270181|NULL|     F|        2|
|     268073|NULL|     M|        1|
|     273216|NULL|     F|        5|
|     266929|NULL|     M|        8|
|     275152|NULL|     M|        4|
|     275034|NULL|     F|        4|
|     273966|NULL|     M|        8|
|     270829|NULL|     F|        8|
|     266997|NULL|     M|        9|
|     272455|NULL|     M|        7|
|     274593|NULL|     M|       10|
|     273391|NULL|     F|        2|
|     267657|NULL|     F|        7|
|     271344|NULL|     F|        5|
|     268275|NULL|     F|        7|
|     274630|NULL|     M|        4|
|     273541|NULL|     M|        4|
+-----------+----+------+---------+
only showing top 20 rows



In [29]:
customer_df.join(transaction,[customer_df.customer_Id == transaction.cust_id],"inner").toPandas()

,customer_Id,DOB,Gender,city_code,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,270351,None,M,5.0,80712190438,270351,None,1,1,-5,-772,405.300,-4265.300,e-Shop
1,270384,None,F,8.0,29258453508,270384,None,5,3,-5,-1497,785.925,-8270.925,e-Shop
2,273420,None,M,8.0,51750724947,273420,None,6,5,-2,-791,166.110,-1748.110,TeleShop
3,271509,None,M,3.0,93274880719,271509,None,11,6,-3,-1363,429.345,-4518.345,e-Shop
4,273420,None,M,8.0,51750724947,273420,None,6,5,-2,-791,166.110,-1748.110,TeleShop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23048,274550,None,M,7.0,94340757522,274550,None,12,5,1,1264,132.720,1396.720,e-Shop
23049,270022,None,M,9.0,89780862956,270022,None,4,1,1,677,71.085,748.085,e-Shop
23050,271020,None,M,8.0,85115299378,271020,None,2,6,4,1052,441.840,4649.840,MBR
23051,270911,None,M,2.0,72870271171,270911,None,11,5,3,1142,359.730,3785.730,TeleShop


In [47]:
spark.stop()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
/opt/spark/python/pyspark/context.py:657: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  warnings.warn(


ConnectionRefusedError: [Errno 111] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
